The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

In [ ]:

mkdir -p ~/students

wget -nc -P ~/students https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

In [ ]:
$students = Import-Csv ~/students/LCP_22-23_students.csv

$students | Where-Object { $_.Department -eq 'PoD' } | Export-Csv -Path ~/students/PoD_students.csv -NoTypeInformation
$students | Where-Object { $_.Department -eq 'Physics' } | Export-Csv -Path ~/students/Physics_students.csv -NoTypeInformation

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

In [ ]:
$alphabetCounts = @{}

foreach ($letter in 'A'..'Z') {
    $count = ($students | Where-Object { $_.Surname -like "$letter*" }).Count
    $alphabetCounts[$letter] = $count
}

# Output the counts
$alphabetCounts

1\.d Find out which is the letter with most counts.

In [ ]:
$maxLetter = $alphabetCounts.GetEnumerator() | Sort-Object -Property Value -Descending | Select-Object -First 1


$maxLetter

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

In [ ]:
$students | ForEach-Object -Begin { $i = 1 } -Process {
    $group = ($i % 18) + 1
    $_ | Export-Csv -Path ~/students/group_$group.csv -Append -NoTypeInformation
    $i++
}

### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

In [ ]:
# Remove metadata and commas between numbers in data.csv and save it as data.txt
Get-Content data.csv | ForEach-Object {
    $_ -replace ',','' -replace 'metadata','' | Out-File -Append data.txt
}

2\.b How many even numbers are there?

In [ ]:
$evenCount = 0
Get-Content data.txt | ForEach-Object {
    $numbers = $_ -split ' '
    foreach ($number in $numbers) {
        if ([int]$number % 2 -eq 0) {
            $evenCount++
        }
    }
}

$evenCount

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

In [ ]:
$threshold = 100 * [math]::sqrt(3) / 2
$group1Count = 0
$group2Count = 0

Get-Content data.txt | ForEach-Object {
    $numbers = $_ -split ' '
    $x = [double]$numbers[0]
    $y = [double]$numbers[1]
    $z = [double]$numbers[2]
    $distance = [math]::sqrt($x * $x + $y * $y + $z * $z)
    
    if ($distance -gt $threshold) {
        $group1Count++
    } else {
        $group2Count++
    }
}

"Group 1 (distance > $threshold): $group1Count"
"Group 2 (distance <= $threshold): $group2Count"

2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

In [ ]:
param (
    [int]$n
)

for ($i = 1; $i -le $n; $i++) {
    $outputFile = "data_copy_$i.txt"
    Get-Content data.txt | ForEach-Object {
        $numbers = $_ -split ' '
        $newNumbers = $numbers | ForEach-Object { [double]$_ / $i }
        $newNumbers -join ' ' | Out-File -Append $outputFile
    }
}